Notebook adapted from: https://github.com/jfpuget/p-hacking/blob/master/Green-dice-p-value-hacking.ipynb

# P-Hacking

It demonstrates how to mislead people about a scientific experiment by hacking p-values following the methodology from this [xkcd comic](http://xkcd.com/882/):

<img src="http://imgs.xkcd.com/comics/significant.png"></img>

First, some useful imports.

In [1]:
import numpy as np
import pandas as pd
from numpy.random import randint, seed

## The experiment

We are given 20 dice colors, and we roll dice for each color 1,000 times.  We report the number of six.
We set the random seed to make sure results are reproducible.

In [2]:
dice = ['Purple', 'Brown', 'Pink', 'Blue', 'Teal', 
        'Salmon', 'Red', 'Turquoise', 'Magenta', 'Yellow', 
        'Grey', 'Tan', 'Cyan', 'Green', 'Mauve',
        'Beige', 'Lilac', 'Black', 'Peach', 'Orange']

def dice_experiments(dice, n):
    df = pd.DataFrame(index=dice)
    for die in dice:         
        result = randint(1,6+1,size=n)
        #counts how many times 6 was the outcome
        df.loc[die,'Number of Six'] = np.sum(result[result==6])//6
    return df

np.random.seed(75000)
df = dice_experiments(dice, 1000)
df

,Number of Six
Purple,151.0
Brown,167.0
Pink,158.0
Blue,167.0
Teal,181.0
Salmon,162.0
Red,170.0
Turquoise,161.0
Magenta,165.0
Yellow,180.0


The green color has 188 times a six, which is rather high. Indeed, average value is 1,000/6 = 167.

The probability that green dice have at least 188 six is about 4% as we will see below.  This is the *p-value* for the green color result. If we decided to watch the green color before the experiment, then this low p-value would warrant us a scientific publication.  

Let's see how unlikely this result is by running our experiment many times.

In [3]:
def get_fraction_one_color(color, dice, k, n, repeat):
    success = 0.0
    for experiment in range(repeat):
        df = dice_experiments(dice, n)
        if df.loc[color,'Number of Six'] >= k:
            success += 1
    return success/repeat

get_fraction_one_color('Green', dice, 188, 1000, 1000)

0.041

This is about 4.1%, i.e. rather unlikely. There must be a reason why green dice favor six that much!

<h1>Do you agree with the analysis?</h1>

<h1>Is a low p-value good enough for this scenario?</h1>


## The Catch

We misinterpreted what we did and our conclusion is misleading.  What we did was to run the experiment for 20 colors, then select the color with the highest number of six.  The p-value should be the probability that at least one color gets at least 188 six. 

Let's evaluate this probability experimentally.

In [8]:
def get_fraction_any_color(dice, k, n, repeat):
    success = 0.0
    for experiment in range(repeat):
        df = dice_experiments(dice, n)
        if df['Number of Six'].max() >= k:
            success += 1
    return success/repeat

get_fraction_any_color(dice, 188, 1000, 1000)

0.561

In [9]:
df

,Number of Six
Purple,151.0
Brown,167.0
Pink,158.0
Blue,167.0
Teal,181.0
Salmon,162.0
Red,170.0
Turquoise,161.0
Magenta,165.0
Yellow,180.0


This probability is about 56%, i.e. very likely.  There is nothing special about our dice afterall.

This little code may look contrived, but it is not.  scientists got publications using a similar methodology, see the pdf for details.

## Ultimate p-hacking

We wanted to mimick the xkcd comic, hence we looked for a random seed that yields the result we want.  This is really bad p-hacking, as we define first the p-value, then find experiment settings that produce it. 

In [5]:
def find_seed(dice, k, n, repeat, rounding):
    nseed = 0.0
    for seed in range(0,repeat,rounding):
        np.random.seed(seed)
        df = dice_experiments(dice, n)
        m = df['Number of Six'].max()
        if m == k and m == df.loc['Green','Number of Six']:
            return seed
    
res = find_seed(dice, 188, 1000, 1000000, 1000)
res

75000